<a href="https://colab.research.google.com/github/eleron7/Daegu_Complain_Classifficaion/blob/main/%EB%B6%80%EC%84%9C%EB%B3%84_%EC%B5%9C%EB%B9%88%EA%B0%92_%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**‼처음부터 실행하지 말고, 각 파트 끝에 데이터를 저장하고, 다음 파트 담당자는 해당 데이터를 불러와서 작업할 것**
- 각 load, save 함수 사용

In [ ]:
name_list = ['BukGu', 'DalseongGun', 'DalsuGu', 'DongGu', 'JungGu', 'NamGu', 'SeoGu', 'Suseonggu']
complain_list = []

In [ ]:
def save():
    for i, v in enumerate(name_list):
        complain_list[i].to_csv(v + '.csv',index=False)

In [ ]:
def load():
    for i, v in enumerate(name_list):
        complain_list.append(pd.read_csv(v + '.csv'))

# 0. 라이브러리, row data 불러오기

In [ ]:
import numpy as np
import pandas as pd

----

## 7.부서별 최빈 단어 찾기


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
name_list

['BukGu',
 'DalseongGun',
 'DalsuGu',
 'DongGu',
 'JungGu',
 'NamGu',
 'SeoGu',
 'Suseonggu']

In [ ]:
def load_for_corab():
    for i, v in enumerate(name_list):
        complain_list.append(pd.read_csv('/content/drive/MyDrive/벨류업 프로젝트 공모전/alldata/'+ v + '.csv'))

In [ ]:
load_for_corab()

In [ ]:
#그냥 날려버리기
complain_list[3].drop(2262,axis=0,inplace=True)
complain_list[6].drop(1625,axis=0,inplace=True)

In [ ]:
complain_list[3].drop('index',axis=1,inplace=True)
complain_list[6].drop('index',axis=1,inplace=True)

KeyError: ignored

In [ ]:
complain_list[3].reset_index(inplace=True)
complain_list[6].reset_index(inplace=True)

In [ ]:
complain_list[6]

,content,dept
0,매번 수거 하시는 분들만 수고하시네요 동일고충민원여부동일고충민원여부 선행민원번호 ...,읍/면/동사무소
1,비대면 처방전 거부한 약국 있습니다 아이가 콧물 감기로 아프기시작해서비대면 처방을받...,보건소 보건행정과
2,전봇대 우째 좀 안됩니까 가 설치 되어도 무단 투기가 개선 되지 않는 건 전봇대가...,읍/면/동사무소
3,평리구역서대구 영무예다음 부실 시공 등 시공사 소극적 조치에 대한 민원 우리 서구의...,도시안전국 건축주택과
4,여름철대비 비산동에 사는 주민입니다 공무에 힘쓰시느라 수고가 많으십니다 비산동이 주...,보건소 보건행정과
...,...,...
2369,저 저번에 질문한 사람인데요 죄송하지만 우편물이 또 그사람 이름으로 날라왔거든요그일...,읍/면/동사무소
2370,뭐좀 물어볼려구요 저기요집으로 다른사람이름으로 음식물쓰레기 수수료 국민 건강보험 독...,읍/면/동사무소
2371,소음 및 분진 피해삼육비철 구정에 고생이 많으십니다저는 서구 원대가 번지에 처와 개...,사회산업국 환경관리과
2372,서구청 귀청의 전력 에너지 사용량 자료 요청 귀청의 무한한 발전을 기원합니다안녕하십...,도시국 건축주택과


In [ ]:
#형태소 분석할  after열 생성
for i in range(0,len(complain_list)):
  complain_list[i]['after']=complain_list[i]['content'].copy()

In [ ]:
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 46.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.7 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=de149afa669828aeb90718a806c96ae47aa26f6b4f96803587a807a5128dfd17
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from konlpy.tag import *
mecab = MeCab()
# hannanum = Hannanum()

In [ ]:
mecab.nouns('나는 배가 고파서 햄버거 조졌다,이것은 햄버거야')

['나', '배', '햄버거', '이것', '햄버거']

In [ ]:
complain_list[0].head(10)

,content,dept,after
0,매천동 청구장미아파트삼우그린 입구 불합리 위치 매천동 청구장미아파트삼우그린아파트 ...,도시국 건설과,"[매천동, 청구장미아파트삼우그린, 입구, 불합리, 위치, 매천동, 청구장미아파트삼우..."
1,처리중언제까지 어린이보호구역 불법주차 불법적재 신고한지가 언제인데 아직 처리중적재물...,도시국 도시행정과,"[처리중언제, 어린이보호구역, 불법주차, 불법적재, 지, 언제, 처리중적재물, 그대..."
2,도로개설 본인은 대구시 북구 태전동 번지 토지 소유자 입니다현재 상기 토지에서 ...,도시국 건설과,"[도로개설, 본인, 대구시, 북구, 태전동, 토지, 소유자, 입니다현재, 상기, 토..."
3,북구청장감사실은 뭐 하는 곳 입니까 묻습니다 불법에 대한 최고의 북구 전문가 북구청...,감사실,"[북구청장감사실, 뭐, 곳, 불법, 최고, 북구, 전문가, 북구청장감사실, 뭐, 곳..."
4,대구시장 북구청장 감사실은 뭐 하는 부서 입니까 감사실 폐쇄하셔요 미관지구 대구시장...,감사실,"[대구시장, 북구청장, 감사실, 뭐, 부, 감사, 폐쇄, 미관지구, 대구시장, 북구..."
5,북구청장 주거환경정비 해당주민 반대에 전화 문자 카톡 구민재산 생활 고통종식하라 본...,도시국 건축주택과,"[북구청장, 주거환경정비, 해당주민, 반대, 전화, 문자, 카톡, 구민재산, 생활,..."
6,북구청장 재활용쓰레기 처리 못 하는게 경영의 최고 전문가 인가 평가는 구민이 북구청...,복지환경국 자원순환과,"[북구청장, 재활용쓰레기, 처리, 경영, 최고, 전문, 평, 구민, 북구청장감사실,..."
7,북구청장 최고의 전문가 행정이 북구민에게 어디까지 미치나요 답변해 주셔요 북구청장북...,감사실,"[북구청장, 최고, 전문, 행정, 북구민, 어디, 답변, 북구청장북구청비서실, 구민..."
8,칠성초 횡단보도부근 및 침산동 일대 공사마무리를 서둘러주시기 바랍니다 침산동 국민은...,도시국 건설과,"[칠성초, 횡단보도부근, 침산동, 일대, 공사마무리, 침산동, 국민은행부근, 하수관..."
9,에어간판으로 인한 교통사고유발 민원을 넣었지만 해결되지 않았습니다 동일고충민원여부동...,도시국 도시행정과,"[판, 교통사고유발, 민원, 해결, 동일고충민원여부동일고충민원여부, 선행민원번호, ..."


In [ ]:
#명사만 추출해버리기
error_list=[]
for i in range(0,len(complain_list)):
  for j in range(0,len(complain_list[i]['after'])):
    complain_list[i]['after'][j]=mecab.nouns(complain_list[i]['content'][j])
   

In [ ]:
#각 지역+각 부서명을 변수이름으로 하는 리스트들을 만들어 거기에 명사들을 넣어주자
complain_list[0]['dept'].unique()

array(['도시국 건설과', '도시국 도시행정과', '감사실', '도시국 건축주택과', '복지환경국 자원순환과',
       '복지환경국 환경관리과', '도시국 교통과', '문화녹지국 공원녹지과', '읍/면/동사무소', '도시국 도시관리과',
       '주민생활지원국 환경관리과', '도시국 재난안전과'], dtype=object)

In [ ]:
#dept에 띄어쓰기가 있어서 리스트 명으로 못쓰니까 띄어쓰기를 '_'로 바꿔줄게요

for i in range(0,len(complain_list)):
  complain_list[i]['after_dept']=complain_list[i]['dept'].copy()

error_list=[]
for j in range(0,len(complain_list)):
  for k in range(0,len(complain_list[j]['after_dept'])):
    try:
      complain_list[j]['after_dept'][k]=complain_list[j]['after_dept'][k].replace(" ","_")
    except:
      error_list.append([j,k])
      continue
    

In [ ]:
complain_list[0].head(10)

,content,dept,after,after_dept
0,매천동 청구장미아파트삼우그린 입구 불합리 위치 매천동 청구장미아파트삼우그린아파트 ...,도시국 건설과,"[매천동, 청구, 장미, 아파트, 삼우, 그린, 입구, 합리, 위치, 매천동, 청구...",도시국_건설과
1,처리중언제까지 어린이보호구역 불법주차 불법적재 신고한지가 언제인데 아직 처리중적재물...,도시국 도시행정과,"[처리, 중, 언제, 어린이, 보호구역, 법주, 차, 불법, 신고, 지, 언제, 처...",도시국_도시행정과
2,도로개설 본인은 대구시 북구 태전동 번지 토지 소유자 입니다현재 상기 토지에서 ...,도시국 건설과,"[도로, 개설, 본인, 대구시, 북구, 태전동, 번지, 토지, 소유자, 상기, 토지...",도시국_건설과
3,북구청장감사실은 뭐 하는 곳 입니까 묻습니다 불법에 대한 최고의 북구 전문가 북구청...,감사실,"[북, 구청장, 사실, 곳, 불법, 최고, 북구, 전문가, 북, 구청장, 사실, 곳...",감사실
4,대구시장 북구청장 감사실은 뭐 하는 부서 입니까 감사실 폐쇄하셔요 미관지구 대구시장...,감사실,"[대구, 시장, 북, 구청장, 감, 사실, 부서, 폐쇄, 미관, 지구, 대구, 시장...",감사실
5,북구청장 주거환경정비 해당주민 반대에 전화 문자 카톡 구민재산 생활 고통종식하라 본...,도시국 건축주택과,"[북, 구청장, 주거, 환경, 정비, 해당, 주민, 반대, 전화, 문자, 카톡, 구...",도시국_건축주택과
6,북구청장 재활용쓰레기 처리 못 하는게 경영의 최고 전문가 인가 평가는 구민이 북구청...,복지환경국 자원순환과,"[북, 구청장, 활용, 쓰레기, 처리, 게, 경영, 최고, 전문가, 평가, 구민, ...",복지환경국_자원순환과
7,북구청장 최고의 전문가 행정이 북구민에게 어디까지 미치나요 답변해 주셔요 북구청장북...,감사실,"[북, 구청장, 최고, 전문가, 행정, 북, 구민, 어디, 답변, 북구, 청장, 북...",감사실
8,칠성초 횡단보도부근 및 침산동 일대 공사마무리를 서둘러주시기 바랍니다 침산동 국민은...,도시국 건설과,"[칠성, 초, 횡단보도, 부근, 침산동, 일대, 공사, 마무리, 침산동, 국민은행,...",도시국_건설과
9,에어간판으로 인한 교통사고유발 민원을 넣었지만 해결되지 않았습니다 동일고충민원여부동...,도시국 도시행정과,"[간판, 교통사고, 유발, 민원, 해결, 동일, 고충, 민원, 여부, 동일, 고충,...",도시국_도시행정과


In [ ]:
# globals()['{}_{}'.format(name_list[0],complain_list[0]['after_dept'].unique()[0])]=[]

In [ ]:
#각 지역+부서를 변수 이름으로 하는 리스트 만들기
my_list=[]
error_list=[]
for i in range(0,len(complain_list)):
  for j in range(0,len(complain_list[i]['after_dept'].unique())):
    try:
      globals()['{}_{}'.format(name_list[i],complain_list[i]['after_dept'].unique()[j])]=[]
      tmp_str='{}_{}'.format(name_list[i],complain_list[i]['after_dept'].unique()[j])
      my_list.append(tmp_str)
    except:
      error_list.append([i,j])
      continue


In [ ]:
my_list

['BukGu_도시국_건설과',
 'BukGu_도시국_도시행정과',
 'BukGu_감사실',
 'BukGu_도시국_건축주택과',
 'BukGu_복지환경국_자원순환과',
 'BukGu_복지환경국_환경관리과',
 'BukGu_도시국_교통과',
 'BukGu_문화녹지국_공원녹지과',
 'BukGu_읍/면/동사무소',
 'BukGu_도시국_도시관리과',
 'BukGu_주민생활지원국_환경관리과',
 'BukGu_도시국_재난안전과',
 'DalseongGun_건설도시국_건축과',
 'DalseongGun_건설도시국_안전총괄과',
 'DalseongGun_문화관광국_공원녹지과',
 'DalseongGun_건설도시국_건설과',
 'DalseongGun_경제환경국_교통과',
 'DalseongGun_교육복지국_교육정책과',
 'DalseongGun_경제환경국_환경과',
 'DalseongGun_건설도시국_도시정비과',
 'DalseongGun_읍/면/동사무소',
 'DalseongGun_자치행정국_종합민원과',
 'DalseongGun_경제환경국_청소위생과',
 'DalseongGun_문화관광국_문화예술과',
 'DalseongGun_경제환경국_농업정책과',
 'DalseongGun_보건소_보건과',
 'DalseongGun_건설도시국_도시계획과',
 'DalseongGun_건설도시국_교통과',
 'DalsuGu_도시창조국_공원녹지과',
 'DalsuGu_경제환경국_청소과',
 'DalsuGu_경제환경국_주차관리과',
 'DalsuGu_경제환경국_교통행정과',
 'DalsuGu_도시창조국_건설과',
 'DalsuGu_도시창조국_건축과',
 'DalsuGu_경제환경국_기후환경과',
 'DalsuGu_읍/면/동사무소',
 'DalsuGu_보건소_보건행정과',
 'DalsuGu_경제환경국_환경보호과',
 'DalsuGu_도시창조국_도시재생과',
 'DalsuGu_복지문화국_문화체육관광과',
 'DalsuGu_자치행정국_총무과',
 'DalsuGu_경제환경국_일자리경제과',
 'Da

In [ ]:
#각 과의 이름을 가진 리스트에 요소들 다 추가

for i in range(0,len(complain_list)):#각 구에 대해

  for j in range(0,len(complain_list[i]['after_dept'])):# 각 구의 데이터 프레임에 after_dept열에 대해

    for k in range(0,len(complain_list[i]['after'][j])):#각 구의 데이터 프레임에 after_dept열의 요소들에 대해
      if complain_list[i]['after'][j][k] in ['것','이것','저것','그것']:#한번 돌려봤는데 최빈값이 싹다 "것"이여서 제거
        continue
      else:
        (globals()['{}_{}'.format(name_list[i],complain_list[i]['after_dept'][j])]).append(complain_list[i]['after'][j][k])
   


In [ ]:
BukGu_도시국_건설과

['매천동',
 '청구',
 '장미',
 '아파트',
 '삼우',
 '그린',
 '입구',
 '합리',
 '위치',
 '매천동',
 '청구',
 '장미',
 '아파트',
 '삼우',
 '아파트',
 '입구',
 '매천',
 '생활',
 '용품',
 '옆',
 '진입로',
 '내용',
 '사람',
 '이용',
 '횡단',
 '보도',
 '아파트',
 '차량',
 '다님',
 '새벽',
 '매천',
 '생활',
 '용품',
 '점',
 '자동차',
 '수리',
 '점',
 '문',
 '시간',
 '거리',
 '보행자',
 '우려',
 '자동차',
 '우측',
 '사각지대',
 '밤',
 '거리',
 '매천',
 '생활',
 '용품',
 '점',
 '자동차',
 '수리',
 '점',
 '문',
 '시간',
 '거리',
 '조명',
 '시설',
 '확인',
 '후',
 '조치',
 '사진',
 '첨부',
 '도로',
 '개설',
 '본인',
 '대구시',
 '북구',
 '태전동',
 '번지',
 '토지',
 '소유자',
 '상기',
 '토지',
 '농사',
 '본인',
 '외',
 '필지',
 '소유자',
 '통행로',
 '한정',
 '범위',
 '내',
 '영농',
 '수',
 '밖',
 '민법',
 '제조',
 '주위',
 '토지',
 '토지',
 '건너',
 '태전동',
 '번지',
 '국토',
 '교통부',
 '소유',
 '도로',
 '현황',
 '농림',
 '축산',
 '부',
 '소유',
 '태전동',
 '번지',
 '구거',
 '잠식',
 '수풀',
 '도로',
 '역활',
 '이',
 '민원인',
 '상기',
 '번지',
 '도로',
 '개설',
 '부탁',
 '칠성',
 '초',
 '횡단보도',
 '부근',
 '침산동',
 '일대',
 '공사',
 '마무리',
 '침산동',
 '국민은행',
 '부근',
 '하수관',
 '공사',
 '얼마',
 '전',
 '땅',
 '관',
 '공사',
 '자체',
 '아이',
 '통행',
 '위험',
 '공사',
 '

In [ ]:
from collections import Counter
#최빈값을 10개까지!
counter = Counter(BukGu_도시국_건설과)
most_common = counter.most_common()

In [ ]:
most_common[:10]

[('도로', 1197),
 ('수', 1123),
 ('건설', 966),
 ('민원', 875),
 ('분할', 800),
 ('답변', 740),
 ('공사', 726),
 ('설치', 697),
 ('당시', 641),
 ('인도', 583)]

In [ ]:
#각 구별 부서 리스트의 값을 최빈값 10개로 바꾸자!
for i in range(0,len(my_list)):
  counter = Counter(globals()['{}'.format(my_list[i])])
  most_common = counter.most_common()
  globals()['{}'.format(my_list[i])]=most_common[:10]

In [ ]:
#이렇게 바뀜
for i in range(0,len(my_list)):
  print (globals()['{}'.format(my_list[i])])

[('도로', 1197), ('수', 1123), ('건설', 966), ('민원', 875), ('분할', 800), ('답변', 740), ('공사', 726), ('설치', 697), ('당시', 641), ('인도', 583)]
[('불법', 108), ('단속', 97), ('민원', 97), ('물', 80), ('광고', 72), ('북구', 63), ('시장', 57), ('인도', 55), ('앞', 52), ('노점', 51)]
[('불법', 6189), ('시장', 4369), ('건축', 4052), ('북', 3996), ('행정', 3920), ('구민', 3564), ('구청장', 3173), ('북구', 2863), ('대구', 2806), ('허가', 2656)]
[('수', 1784), ('시설', 1763), ('제조', 1753), ('건축', 1738), ('아파트', 1493), ('제호', 1478), ('등', 1437), ('지역', 1259), ('건축물', 1232), ('별표', 1152)]
[('쓰레기', 817), ('분리수거', 185), ('행정', 182), ('수거', 172), ('처리', 169), ('활용', 168), ('북구', 167), ('민원', 164), ('투기', 155), ('수', 144)]
[('소음', 478), ('공사', 467), ('쓰레기', 454), ('수', 269), ('민원', 247), ('주민', 230), ('아파트', 182), ('현장', 176), ('집', 166), ('북구', 161)]
[('주차', 3408), ('단속', 2172), ('차량', 1774), ('차', 1628), ('불법', 1253), ('도로', 1156), ('수', 841), ('앞', 799), ('민원', 632), ('아파트', 609)]
[('공원', 285), ('나무', 131), ('수', 106), ('설치', 99), ('북구', 94), ('주민

In [ ]:
#가독성을 위해 딕셔너리 형태로
final_dic={}
for i in range(0,len(my_list)):
  final_dic[my_list[i]]=globals()['{}'.format(my_list[i])]

In [ ]:
final_dic

{'BukGu_도시국_건설과': [('도로', 1197),
  ('수', 1123),
  ('건설', 966),
  ('민원', 875),
  ('분할', 800),
  ('답변', 740),
  ('공사', 726),
  ('설치', 697),
  ('당시', 641),
  ('인도', 583)],
 'BukGu_도시국_도시행정과': [('불법', 108),
  ('단속', 97),
  ('민원', 97),
  ('물', 80),
  ('광고', 72),
  ('북구', 63),
  ('시장', 57),
  ('인도', 55),
  ('앞', 52),
  ('노점', 51)],
 'BukGu_감사실': [('불법', 6189),
  ('시장', 4369),
  ('건축', 4052),
  ('북', 3996),
  ('행정', 3920),
  ('구민', 3564),
  ('구청장', 3173),
  ('북구', 2863),
  ('대구', 2806),
  ('허가', 2656)],
 'BukGu_도시국_건축주택과': [('수', 1784),
  ('시설', 1763),
  ('제조', 1753),
  ('건축', 1738),
  ('아파트', 1493),
  ('제호', 1478),
  ('등', 1437),
  ('지역', 1259),
  ('건축물', 1232),
  ('별표', 1152)],
 'BukGu_복지환경국_자원순환과': [('쓰레기', 817),
  ('분리수거', 185),
  ('행정', 182),
  ('수거', 172),
  ('처리', 169),
  ('활용', 168),
  ('북구', 167),
  ('민원', 164),
  ('투기', 155),
  ('수', 144)],
 'BukGu_복지환경국_환경관리과': [('소음', 478),
  ('공사', 467),
  ('쓰레기', 454),
  ('수', 269),
  ('민원', 247),
  ('주민', 230),
  ('아파트', 182),
  ('현장', 176),
  

In [ ]:
#데이터 프레임으로!
mode_df=pd.DataFrame(final_dic)

In [ ]:
mode_df.head(10)

,BukGu_도시국_건설과,BukGu_도시국_도시행정과,BukGu_감사실,BukGu_도시국_건축주택과,BukGu_복지환경국_자원순환과,BukGu_복지환경국_환경관리과,BukGu_도시국_교통과,BukGu_문화녹지국_공원녹지과,BukGu_읍/면/동사무소,BukGu_도시국_도시관리과,...,Suseonggu_도시국_교통과,Suseonggu_도시국_건축과,Suseonggu_복지국_녹색환경과,Suseonggu_도시국_공원녹지과,Suseonggu_읍/면/동사무소,Suseonggu_도시국_도시디자인과,Suseonggu_보건소_보건행정과,Suseonggu_복지국_경제환경과,Suseonggu_교육문화국_문화체육과,Suseonggu_복지국_복지과
0,"(도로, 1197)","(불법, 108)","(불법, 6189)","(수, 1784)","(쓰레기, 817)","(소음, 478)","(주차, 3408)","(공원, 285)","(주민, 215)","(공원, 442)",...,"(주차, 3296)","(수, 3384)","(소음, 795)","(공원, 803)","(주민, 116)","(도로, 121)","(모기, 115)","(소음, 735)","(도서관, 343)","(수, 105)"
1,"(수, 1123)","(단속, 97)","(시장, 4369)","(시설, 1763)","(분리수거, 185)","(공사, 467)","(단속, 2172)","(나무, 131)","(일, 137)","(수, 275)",...,"(단속, 2138)","(수성, 3312)","(공사, 719)","(수성, 534)","(쓰레기, 74)","(주민, 109)","(보건소, 91)","(공사, 687)","(수, 326)","(어린이집, 96)"
2,"(건설, 966)","(민원, 97)","(건축, 4052)","(제조, 1753)","(행정, 182)","(쓰레기, 454)","(차량, 1774)","(수, 106)","(직원, 130)","(현수막, 224)",...,"(차량, 2034)","(아파트, 3251)","(수성, 370)","(수, 508)","(일, 70)","(수, 84)","(방역, 79)","(수, 466)","(수성, 213)","(아이, 88)"
3,"(민원, 875)","(물, 80)","(북, 3996)","(건축, 1738)","(수거, 172)","(수, 269)","(차, 1628)","(설치, 99)","(센터, 127)","(나무, 222)",...,"(차, 1645)","(분양, 2988)","(민원, 361)","(설치, 376)","(수, 64)","(불법, 84)","(수, 75)","(아파트, 297)","(사람, 206)","(장애, 84)"
4,"(분할, 800)","(광고, 72)","(행정, 3920)","(아파트, 1493)","(처리, 169)","(민원, 247)","(불법, 1253)","(북구, 94)","(사람, 125)","(주민, 212)",...,"(도로, 1497)","(공사, 2785)","(수, 332)","(주민, 336)","(가로등, 60)","(구청, 82)","(수성구, 57)","(민원, 273)","(테니스장, 206)","(등, 81)"
5,"(답변, 740)","(북구, 63)","(구민, 3564)","(제호, 1478)","(활용, 168)","(주민, 230)","(도로, 1156)","(주민, 91)","(말, 115)","(북구, 203)",...,"(불법, 1343)","(구청, 2489)","(현장, 314)","(등, 335)","(센터, 54)","(수성, 80)","(벌레, 42)","(소리, 269)","(운영, 192)","(구청, 72)"
6,"(공사, 726)","(시장, 57)","(구청장, 3173)","(등, 1437)","(북구, 167)","(아파트, 182)","(수, 841)","(공사, 89)","(수, 111)","(가로수, 186)",...,"(앞, 945)","(입주, 2272)","(구청, 224)","(사람, 317)","(민원, 51)","(민원, 74)","(검사, 39)","(구청, 265)","(시간, 187)","(수성, 69)"
7,"(설치, 697)","(인도, 55)","(북구, 2863)","(지역, 1259)","(민원, 164)","(현장, 176)","(앞, 799)","(불편, 88)","(제, 103)","(등, 176)",...,"(수, 911)","(승인, 1788)","(먼지, 223)","(생각, 295)","(분, 49)","(인도, 71)","(때문, 38)","(주민, 261)","(이용, 180)","(고산, 66)"
8,"(당시, 641)","(앞, 52)","(대구, 2806)","(건축물, 1232)","(투기, 155)","(집, 166)","(민원, 632)","(가로수, 86)","(동사무소, 97)","(사람, 166)",...,"(주정차, 803)","(일, 1528)","(진동, 222)","(구청, 247)","(부탁, 47)","(앞, 68)","(수성, 34)","(집, 253)","(생각, 175)","(학교, 62)"
9,"(인도, 583)","(노점, 51)","(허가, 2656)","(별표, 1152)","(수, 144)","(북구, 161)","(아파트, 609)","(앞, 83)","(저, 93)","(일, 158)",...,"(설치, 780)","(문제, 1521)","(주민, 216)","(나무, 241)","(앞, 47)","(현수막, 68)","(병원, 33)","(현장, 233)","(주민, 155)","(원, 61)"


In [ ]:
# mode_df.to_csv('mode_data.csv',index=False)
mode_df.to_csv('/content/drive/MyDrive/벨류업 프로젝트 공모전/mode_data.csv',index=False)